# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
df = pd.read_csv("my_data.csv")
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   City         60 non-null     object 
 1   Country      55 non-null     object 
 2   Latitude     55 non-null     float64
 3   Longitude    55 non-null     float64
 4   Date         55 non-null     float64
 5   Wind_Speed   55 non-null     float64
 6   Pressure     55 non-null     float64
 7   Humidity     55 non-null     float64
 8   Temperature  55 non-null     float64
 9   Cloudiness   55 non-null     float64
dtypes: float64(8), object(2)
memory usage: 4.8+ KB


In [3]:
df.describe()

,Latitude,Longitude,Date,Wind_Speed,Pressure,Humidity,Temperature,Cloudiness
count,55.000000,55.000000,5.500000e+01,55.000000,55.000000,55.000000,55.000000,55.000000
mean,14.332105,-1.962249,1.644076e+09,8.468727,1015.363636,76.400000,46.453091,48.527273
std,39.797523,102.802710,8.927400e+01,7.470146,11.952628,16.776968,34.000188,40.932127
min,-54.800000,-175.200000,1.644075e+09,0.000000,965.000000,41.000000,-40.320000,0.000000
25%,-24.435150,-81.871100,1.644076e+09,3.430000,1010.500000,66.500000,26.875000,2.000000
50%,22.545500,-9.266700,1.644076e+09,6.260000,1016.000000,77.000000,57.970000,41.000000
75%,41.734100,94.366300,1.644076e+09,9.820000,1022.000000,91.500000,72.600000,96.500000
max,72.786800,169.846400,1.644076e+09,31.090000,1037.000000,100.000000,86.400000,100.000000


In [4]:
#Dropna to remove 5 null value entries within "City"
df2 = df.dropna().reset_index(drop=True)
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55 entries, 0 to 54
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   City         55 non-null     object 
 1   Country      55 non-null     object 
 2   Latitude     55 non-null     float64
 3   Longitude    55 non-null     float64
 4   Date         55 non-null     float64
 5   Wind_Speed   55 non-null     float64
 6   Pressure     55 non-null     float64
 7   Humidity     55 non-null     float64
 8   Temperature  55 non-null     float64
 9   Cloudiness   55 non-null     float64
dtypes: float64(8), object(2)
memory usage: 4.4+ KB


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
#Locate API Key

gmaps.configure(api_key=g_key)

In [6]:
#Defined by "Locations" use longitude and latitude to create a heatmap showing humidity as weight
locations = df2.loc[:, ["Latitude", "Longitude"]]

#Access the Gkey and previous dataframe
fig = gmaps.figure()
heatmap_layer = gmaps.heatmap_layer(
    locations, weights=df2['Humidity'],
    max_intensity=20, point_radius=5
)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
#Create a new dataframe by assigning the old dataframe to a new "sub" name df3
#Refer to df.describe to find ranges of weather conditions listed, temp max and min, wind and cloudiness colomn



mask = (df2.Temperature < 80) & (df2.Temperature > 70) & (df2.Wind_Speed < 10) & (df2.Cloudiness == 0)

good_cities = df2.loc[mask]
good_cities

,City,Country,Latitude,Longitude,Date,Wind_Speed,Pressure,Humidity,Temperature,Cloudiness
13,Kahului,US,20.8947,-156.4700,1.644076e+09,4.61,1017.0,82.0,70.25,0.0
27,Busselton,AU,-33.6500,115.3333,1.644075e+09,3.18,1004.0,73.0,71.04,0.0
40,Key Largo,US,25.0865,-80.4473,1.644076e+09,6.55,1022.0,74.0,77.13,0.0
47,Abu Dhabi,AE,24.4667,54.3667,1.644076e+09,4.61,1011.0,68.0,71.28,0.0


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
#This is complicated: Create new data frame labeled "hotel_df" to hold the following: A column labeled "Hotel Name" search within 5000 meters(why meters..)
#Use google places API for citys coords, find the first hotel results and put in data frame, then plot markers on heatmap.

# Using the above top five from "good_cities" get the lon,lat of Kahului
target_coordinates = f"{20.8947}, {-156.4700}"
target_search = ""

#Search within 5000 meters
target_radius = 5000
target_type = "lodging"

# Make a dict of the above inputs
params = {
    "location": target_coordinates,
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using our params dictionary
response = requests.get(base_url, params=params)

places_data = response.json()

print(places_data["results"][0]["name"])
print(places_data["results"][0]["vicinity"])
print(places_data["results"][0]["rating"])





Maui Seaside Hotel
100 West Kaahumanu Avenue, Kahului
3.8


In [9]:
#Define for parsing data
hotel_names = []
ratings = []
addresses = []
cities = []
countries = []
lats = []
longs = []

#Create a row loop to test before moving forward
for _, row in good_cities.iterrows():
    print(row.City)

    #coordinates within f string
    target_coordinates = f"{row.Latitude}, {row.Longitude}"
    target_search = ""
    #Type is always the location ID
    target_radius = 5000
    target_type = "lodging"

    # set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": g_key
    }

    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    response = requests.get(base_url, params=params)

    places_data = response.json()

    hotel = places_data["results"][0]["name"]
    address = places_data["results"][0]["vicinity"]
    rating = places_data["results"][0]["rating"]
    
    # put it all in a list
    hotel_names.append(hotel)
    ratings.append(rating)
    addresses.append(address)
    cities.append(row.City)
    countries.append(row.Country)
    lats.append(row.Latitude)
    longs.append(row.Longitude)

Kahului
Busselton
Key Largo
Abu Dhabi


In [10]:
#Create the DataFrame needed to create the HeatMap
hotel_df = pd.DataFrame()
hotel_df['City'] = cities
hotel_df['Country'] = countries
hotel_df['Latitude'] = lats
hotel_df['Longitude'] = longs
hotel_df['Hotel'] = hotel_names
hotel_df['Address'] = addresses
hotel_df['Rating'] = ratings

hotel_df




,City,Country,Latitude,Longitude,Hotel,Address,Rating
0,Kahului,US,20.8947,-156.4700,Maui Seaside Hotel,"100 West Kaahumanu Avenue, Kahului",3.8
1,Busselton,AU,-33.6500,115.3333,Observatory Guest House,"7 Brown Street, Busselton",4.3
2,Key Largo,US,25.0865,-80.4473,Bayside Inn Key Largo,"99490 Overseas Highway, Key Largo",3.6
3,Abu Dhabi,AE,24.4667,54.3667,Beach Rotana,10th Street - Abu Dhabi,4.5


In [11]:
# NOTE: Do not change any of the code in this cell
#Had to change {Hotel Name} to {Hotel} and "Lat, Lng" to the full spelling
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [12]:
hotel_df

,City,Country,Latitude,Longitude,Hotel,Address,Rating
0,Kahului,US,20.8947,-156.4700,Maui Seaside Hotel,"100 West Kaahumanu Avenue, Kahului",3.8
1,Busselton,AU,-33.6500,115.3333,Observatory Guest House,"7 Brown Street, Busselton",4.3
2,Key Largo,US,25.0865,-80.4473,Bayside Inn Key Largo,"99490 Overseas Highway, Key Largo",3.6
3,Abu Dhabi,AE,24.4667,54.3667,Beach Rotana,10th Street - Abu Dhabi,4.5


In [13]:
# Add marker layer ontop of heat map

# create layers
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
heatmap_layer = gmaps.heatmap_layer(
    locations, weights=hotel_df['Rating'],
    max_intensity=20, point_radius=5
)


fig = gmaps.figure()

# add layers
fig.add_layer(marker_layer)
fig.add_layer(heatmap_layer)


fig
# Display figure


Figure(layout=FigureLayout(height='420px'))